In [35]:
# style_1 = """should be simpler and use fewer technical terms compared to the original input"""
# style_2 = """is in a simple and Easy to understand language"""
# style_3 = """should include additional points or details, expanding the content to increase its overall length and provide more information"""
# style_4 = """include a summary that explicitly lists the topics covered in the text at the end of the response."""
# style_5 = """Include simple mnemonics or memory aids to help the student remember key concepts."""
# style_6 = """explains the text including an analogy of general everyday activities one can easily infer """
# style_7 = """Ensure the inclusion of examples if not already provided, with the quantity ranging from 2 to 5, wherever feasible."""

<!-- messages = prompt_template_1.format_messages(text=input_text, Task=Task, Role=Role, style=style, instructions=instructions, output_format=output_format)
 -->

<h1 style = "color : blue; font-family : verdana">RUNNING ON THE FINAL SAMPLE OF PME DATA</h1>

#### getting parsed response

In [42]:
def parse_Response(response):
   if isinstance(response, str):
       # print(response)
       start_index = response.find("{")
       end_index = response.rfind("}")
       if start_index != -1 and end_index != -1:
           valid_json_content = response[start_index : end_index + 1]
           try:
               JSON_response = json.loads(valid_json_content.replace("\n", ""))
               # append_list_to_file(JSON_response)
               return JSON_response
           except json.JSONDecodeError as e:
               print(f"Error decoding JSON response: {e.__class__.__name__} - {e}\n\n Still trying to work on particular exceptions ...")
               print("Actual Content: ", valid_json_content)
       else:
           print("No valid JSON content found in the response.")
       # time.sleep(50)
   elif isinstance(response, dict):
       return response
   else:
       print("No response message found", type(response))


#### rephrasing function

In [43]:
def rephrasing(summary, style, instructions, output_format, template_string,):
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    prompt_template_1 = ChatPromptTemplate.from_template(template_string)

    particular_message = prompt_template_1.format_messages(
        text=summary, style=style, instructions=instructions, output_format=output_format
    )

    # formatted_system_message = (
    #     f"You are an Educational AI model that assists in rephrasing text based on guidelines. "
    #     f"Your task is to rephrase the given text with the following details:\n"
    #     f"- Style: {style}\n"
    #     f"- Instructions: {instructions}\n"
    #     f"- Output format: {output_format}"
    # )
    # # style-01
    # particular_message = prompt_template_1.format_messages(
    #     system_message=formatted_system_message,
    #     user_message=summary
    # )


    # style - 02

    # particular_message = prompt_template_1.format_messages(
    #     system_message="You are an Educational AI model that assists in rephrasing text based on guidelines.",
    #     user_message=f"Given content: {summary}\n\nRephrase this text with the following guidelines:\n\n- Style: {style}\n- Instructions: {instructions}\n\nOutput format:\n{output_format}"
    # )

    try:
    
        start_time = time.time()
        response = model.invoke(particular_message)
        end_time = time.time()
        print("Latency of Response = ", end_time - start_time)
        rephrasedAns = response.content
        try :
            parsed_output = parse_Response(rephrasedAns)
            if parsed_output:
                parsed_output['summary'] = summary
                return parsed_output
            else:
                raise Exception ("could not rephrase :( ")
        except Exception as e:
            print("Error with extraction: ", type(e).__name__, "–", e, "\n", traceback.format_exc())
            return None
    except Exception as e:
        print("Error with invocation: ", type(e).__name__, "–", e, "\n", traceback.format_exc())
        return {"error": str(e)}



#### defining prompt

In [61]:
def definingPrompt():
    styles_dict = {
        "style_1" : """should be simpler and use fewer technical terms compared to the original input""",
        "style_2" : """is in a simple and Easy to understand language""",
        "style_3" : """should include additional points or details, expanding the content to increase its overall length and provide more information""",
        "style_4" : """include a summary that explicitly lists the topics covered in the text at the end of the response.""",
        "style_5" : """Include simple mnemonics or memory aids to help the student remember key concepts.""",
        "style_6" : """explains the text including an analogy of general everyday activities one can easily infer """,
        "style_7" : """Ensure the inclusion of examples if not already provided, with the quantity ranging from 2 to 5, wherever feasible.""",
    }

    style = styles_dict["style_3"]

    instructions = (
        "Rephrase the provided text such that it is Concise while retaining its original meaning. \n"
        "The generated rephrased answer word length should be around that of the given content. \n"
        "Ensure the rephrased response remains in the context of the provided content without introducing unrelated information. \n"
    )
    output_format = (
        "Provide the output in JSON format with the following key:\n"
        "- `rephrased_text_response`: The rephrased version of the input text."
        "Ensure there is no additional text, commentary, or formatting outside the JSON structure."
    )
    template_string = (
        "You're an Educational AI model designed to Rephrase text, which has all the knowledge of the undergraduate Bachelors of Technology Course."
        "Your task is rephrase the given text based on the your style, the context, and the given guidelines. \n\n"
        "Your writing style : {style}. \n\n"
        "Follow these instructions to generate a good quality rephrased answer: {instructions}\n"
        "Given content to be used for rephrasing: ```{text}```\n\n"
        "{output_format} \n\n"
        "Ensure that you follow all the above guidelines and rules to generate the rephrased content. Any deviation from these guidelines \
        will result in rephrased content not meeting the education standards required for this exercise."
    )
    return style, instructions, output_format, template_string



#### output dataframe file

In [62]:
def output_df(parsedOutputList):
    summAnsList = []
    rephrasedAnsList = []
    for i in parsedOutputList:
        summAnsList.append(i['summary'])
        rephrasedAnsList.append(i['rephrased_text_response'])

    final_df = pd.DataFrame({
                             'summary': summAnsList,
                             'Rephrased Ans' : rephrasedAnsList,
                            })

    return final_df

#### ask_want_to_save_csv()

In [63]:
def ask_want_to_save_csv(final_df):
    import re
    choice = input("Do you want to save the output csv (y/n) : ")
    if choice.lower() == 'y':
        fileName = input("Enter csv file name : ")
        valid_fileName = re.sub(r'[<>:"/\\|?*]', '_', fileName)
        final_df.to_csv(f'{valid_fileName}.csv', index=False)
        print("CSV SAVED")
    else:
        print("Ok no problem")
        print(final_df.head())

#### getting the summarized Answer List from the csv

In [64]:
# # function to load the summarized answer
# def summarizedAnsFunc():
#     df = pd.read_csv('genSum.csv')
#     import ast
#     df['summarized_ans'] = df['summary'].apply(ast.literal_eval)
#     summarizedAnswerList = []
#     for i in df.summarized_ans:
#         summarizedAnswerList.append(i['summarizedAnswer'])

#     # taking input of the range of questions to run for rephrasing
#     print("Enter the range of the question (1-18) from the dataset you want to REPHRASE ")
#     starting_ind = int(input("Starting question index (1-18) : ")) - 1
#     ending_ind = int(input("Ending question index  (1-18) : ")) - 1
#     return summarizedAnswerList[starting_ind: ending_ind]

In [65]:
def summarizedAnsFunc():
    """
    Loads summarized answers from the 'genSum.csv' file and allows the user
    to select a valid range of questions for rephrasing.

    Returns:
        list : A list of summarized answers within the specified range
    """
    import ast

    try:
        # Load the data
        df = pd.read_csv('genSum.csv')
        df['summarized_ans'] = df['summary'].apply(ast.literal_eval)
        summarizedAnswerList = [i['summarizedAnswer'] for i in df.summarized_ans]

        # Validating dataset size
        total_questions = len(summarizedAnswerList)
        print(f"Enter the range of questions (1-{total_questions}) from the dataset you want to REPHRASE")

        # getting the user input with validation
        while True:
            try:
                starting_ind = int(input(f"Starting question index (1-{total_questions}): ")) - 1
                ending_ind = int(input(f"Ending question index (1-{total_questions}): ")) - 1

                if not (0 <= starting_ind < total_questions and 0 <= ending_ind < total_questions):
                    print(f"Indices must be between 1 and {total_questions}. Please try again.")
                elif starting_ind > ending_ind:
                    print("Starting index cannot be greater than the ending index. Please try again.")
                else:
                    break
            except ValueError:
                print("Invalid input. Please enter numeric value only.")

        #loop ended successfully , meaning the range entered is correct
        return summarizedAnswerList[starting_ind: ending_ind + 1]

    except FileNotFoundError:
        print("The file was not found. Please ensure it exists in the current directory.")
    except KeyError as e:
        print(f"Error : Missing column {e} in the dataset. Please check the file format.")
    except Exception as e:
        print(f"An unexpected error occurred in the summarizedAnsFunc() : {e}")


# MAIN FUNCTION

In [66]:
# !pip install langchain_openai langchain

In [67]:
# !pip install python-dotenv

In [68]:
import json
import pandas as pd
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
import getpass
import time

In [69]:
import traceback

In [70]:
def main():
    load_dotenv()

    if not os.environ.get("OPENAI_API_KEY"):
      os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")


    style, instructions, output_format, template_string = definingPrompt() # defining the prompt in this function

    summarizedAnsList = summarizedAnsFunc()  # function to load the summarized answer

    parsedOutputList = []
    # getting rephrasedResponse
    for summary in summarizedAnsList:
        parsed_output = rephrasing(summary, style, instructions, output_format, template_string)
        parsedOutputList.append(parsed_output)


    # make an output csv file
    final_df = output_df(parsedOutputList)
    ask_want_to_save_csv(final_df)


if __name__ == '__main__':
    main()

Enter the range of questions (1-18) from the dataset you want to REPHRASE


Starting question index (1-18):  1
Ending question index (1-18):  4


Latency of Response =  13.998266220092773
Latency of Response =  9.03584623336792
Latency of Response =  8.809804677963257
Latency of Response =  7.722362756729126


Do you want to save the output csv (y/n) :  y
Enter csv file name :  style-3-sample-run


CSV SAVED


# Testing  --- PLEASE IGNORE

In [ ]:
load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

#### prompt

In [ ]:
style, instructions, output_format, template_string = definingPrompt()

In [ ]:
print(style)

should be simpler and use fewer technical terms compared to the original input


In [ ]:
print(instructions)

Rephrase the provided text such that it is Concise while retaining its original meaning. 
The generated rephrased answer word length should be around that of the given content. 
Ensure the rephrased response remains in the context of the provided content without introducing unrelated information. 



In [ ]:
print(output_format)

Provide the output in JSON format with the following keys:
- `input_text`: The original input text.
- `rephrased_text_response`: The rephrased version of the input text.Ensure there is no additional text, commentary, or formatting outside the JSON structure.


In [ ]:
print(template_string)

You're an Educational AI model designed to Rephrase text, which has all the knowledge of the undergraduate Bachelors of Technology Course.Your task is rephrase the given text based on the your style, the context, and the given guidelines. 

Your writing style : {style}. 

Follow these instructions to generate a good quality rephrased answer: {instructions}
Given content to be used for rephrasing: ```{text}```

{output_format} 

Ensure that you follow all the above guidelines and rules to generate the rephrased content. Any deviation from these guidelines         will result in rephrased content not meeting the education standards required for this exercise.


#### summarized ans list

In [ ]:
summarizedAnsList_testing = summarizedAnsFunc()  # function to load the summarized answer

In [ ]:
print(summarizedAnsList_testing)

#### rephrasing test

In [ ]:
def rephrasing_testing(summary, style, instructions, output_format, template_string):
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
    prompt_template_1 = ChatPromptTemplate.from_template(template_string)
    particular_message = prompt_template_1.format_messages(
        text=summary, style=style, instructions=instructions, output_format=output_format
    )
    try:
        response = model.invoke(particular_message)
        rephrasedAns = response.content
        return rephrasedAns
    except Exception as e:
        print(f"An error occurred during rephrasing: {e}")
        return {"error": str(e)}

In [ ]:
resList = []
# for summary in summarizedAnsList_testing:
#         res = rephrasing_testing(summary, style, instructions, output_format, template_string)
#         resList.append(res)
res = rephrasing_testing(summarizedAnsList_testing[0], style, instructions, output_format, template_string)

print(res)

```json
{
    "input_text": "An entrepreneur is a business founder who identifies opportunities, assumes risks, and efficiently uses resources to generate profit and contribute to economic growth. Key characteristics include: \n\n- **Innovation**: Introducing new or improved products/services.\n- **Risk-taking**: Willingness to accept financial, professional, and personal risks.\n- **Vision and Goal Orientation**: Clear long-term goals drive decision-making.\n- **Resilience**: Overcoming hurdles and staying motivated.\n- **Self-Confidence**: Belief in achieving goals and mobilizing resources.\n- **Flexibility**: Adapting strategies to changing market conditions.\n- **Leadership Skills**: Inspiring teams and creating a constructive work environment.\n- **Persistence**: Continually striving to achieve goals despite obstacles.\n- **Opportunity Identification**: Seeing prospects where others see problems.\n- **Financial Acumen**: Understanding budgeting, forecasting, and resource managemen

In [ ]:
parsed_output = parse_Response(res)

In [ ]:
print(parsed_output)

{'input_text': 'An entrepreneur is a business founder who identifies opportunities, assumes risks, and efficiently uses resources to generate profit and contribute to economic growth. Key characteristics include: \n\n- **Innovation**: Introducing new or improved products/services.\n- **Risk-taking**: Willingness to accept financial, professional, and personal risks.\n- **Vision and Goal Orientation**: Clear long-term goals drive decision-making.\n- **Resilience**: Overcoming hurdles and staying motivated.\n- **Self-Confidence**: Belief in achieving goals and mobilizing resources.\n- **Flexibility**: Adapting strategies to changing market conditions.\n- **Leadership Skills**: Inspiring teams and creating a constructive work environment.\n- **Persistence**: Continually striving to achieve goals despite obstacles.\n- **Opportunity Identification**: Seeing prospects where others see problems.\n- **Financial Acumen**: Understanding budgeting, forecasting, and resource management.', 'rephras